In [1]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df


,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,2,26,2024-07-01 19:46:20
1,大安區,21,11,10,2024-07-01 19:43:20
2,大安區,16,7,9,2024-07-01 19:45:21
3,大安區,11,3,8,2024-07-01 19:44:20
4,大安區,16,0,16,2024-07-01 19:47:15
...,...,...,...,...,...
1424,臺大公館校區,30,1,28,2024-07-01 19:47:15
1425,臺大公館校區,20,1,19,2024-07-01 19:47:15
1426,臺大公館校區,24,8,15,2024-07-01 19:39:21
1427,臺大公館校區,40,28,3,2024-07-01 19:47:19


In [10]:
df1 = df.groupby(by='行政區域')[['總數量','可借數量','可還數量']].agg([('車輛數','sum'),('站點數','count')])
df1

總數量       可借數量       可還數量     
         車輛數  站點數   車輛數  站點數   車輛數  站點數
行政區域                                   
中山區     3819  156  1550  156  2243  156
中正區     3209  121  1501  121  1681  121
信義區     3331  106  1292  106  1939  106
內湖區     3499  168  1435  168  2026  168
北投區     2472   94   950   94  1492   94
南港區     2317   92  1108   92  1189   92
士林區     2937  127  1055  127  1818  127
大同區     1629   62   604   62  1016   62
大安區     4987  185  2096  185  2804  185
文山區     2203   97   854   97  1346   97
松山區     2343   88   903   88  1411   88
臺大公館校區  1437   53   499   53   901   53
萬華區     2057   80  1036   80  1000   80